本实验使用Qwen2.5-7B模型来进行未微调的基础测试

### 1. 数据处理 - Tokenizer
#### 1.1 加载数据 - Dataset

In [1]:
import pandas as pd
from datasets import Dataset
dev = pd.read_csv("data/dev.csv")[:100] # 仅前300条数据用于测试
dev_ds = Dataset.from_pandas(dev)

#### 1.2 Tokenization
+ 加载tokenizer 
+ 定义process function：prompt方程
+ 处理dataset为 [input_id, attention_mask,labels]

In [2]:
from modelscope import AutoTokenizer
model_dir ="/root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796" # 定义本地路径
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True,padding_side='left') # 加载tokenizer

In [3]:
# 定义process function:
def process_func(example):
    MAX_LENGTH = 200
    
    instruction = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 
    注意: 1. 如果找到任何实体, 输出必须是严格的json字符串. 如 {'口苦': '临床表现','肺结核': '西医诊断'}。
    2.找不到任何实体时, 输出"没有找到任何实体".
    3.避免输出重复的内容"""
    instructions_messages= [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{example['text']}"}
    ]

    instructions_chatTamplate = tokenizer.apply_chat_template(instructions_messages, tokenize=True, add_generation_prompt=False,return_dict=True)

    input_ids = instructions_chatTamplate['input_ids']
    attention_mask = instructions_chatTamplate['attention_mask']
    
    # 限制最大长度做截断处理
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
    else:
        pad_len = MAX_LENGTH - len(input_ids)
        input_ids = [tokenizer.pad_token_id] * pad_len + input_ids
        attention_mask = [0] * pad_len + attention_mask
    
    return {"input_ids": input_ids, "attention_mask": attention_mask}  

In [4]:
dev_dataset = dev_ds.map(process_func, remove_columns=dev_ds.column_names,num_proc=4) 

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

### 2. 验证模型
#### 2.1 加载原始模型

In [5]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_dir, 
                                            torch_dtype="auto",
                                            device_map="auto")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

测试原始模型

In [6]:
import torch
def predict(dataset, model, tokenizer):
    """
    修正后的预测函数：支持批量/单条样本，处理维度问题，优化切片逻辑
    :param dataset: 处理后的Dataset（如dev_ds_processed[:5]）
    :param model: 加载的LoRA/Qwen2模型
    :param tokenizer: 初始化后的tokenizer
    :return: 模型生成的实体识别结果列表
    """
    # 步骤1：提取数据并转换为二维张量（适配批量输入）
    input_ids = torch.tensor(dataset['input_ids']).to(model.device)
    attention_mask = torch.tensor(dataset['attention_mask']).to(model.device)
    
    # 步骤2：模型生成（禁用梯度计算，节省显存）
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=300,
            temperature=0.7,  # 温度调到0
            # do_sample=False,  # 关闭采样（此时temperature=0等价于贪心搜索）
            repetition_penalty=1.2,  # 加大重复惩罚
            # top_p=0.9,
        )
    
    # 步骤3：修正切片逻辑——去掉输入部分，仅保留模型生成的内容
    # input_ids.shape[1]是单条样本的序列长度（MAX_LENGTH）
    input_seq_len = input_ids.shape[1]
    generated_ids = generated_ids[:, input_seq_len:]  # 批量切片
    
    # 步骤4：解码结果（跳过特殊token，得到纯净文本）
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)    
    # # 打印结果
    # for idx, res in enumerate(response):
    #     print(f"样本{idx+1}生成结果：\n{res}\n" + "-"*80)
    
    return response

In [7]:
import re,ast
def f1_helper(response):
    dataset = {}
    for idx,res in enumerate(response):
        if idx == 16:
            pass
        pattern = r"\{[\s\S]*?\}"  # 匹配JSON大括号片段
        res = re.search(pattern, res)
        
        res = res.group() if res else "{}"
        
        label = dev['label'][idx]
        # print(res,label)
        try:
            res = ast.literal_eval(res)
            res = res if isinstance(res, dict) else {}
        except:
            res = {}
        label = ast.literal_eval(label) # k 是 '腹痛'症状 v 是 '临床表现'


        # 步骤3：正确计算键集（修复核心逻辑颠倒问题）
        label_keys = set(label.keys())
        res_keys = set(res.keys())

        samekeys = label_keys & res_keys  # 标签和预测都有的键（实体匹配）
        diffkeys1 = label_keys - res_keys  # 标签有、预测无 → 漏标（FN）
        diffkeys2 = res_keys - label_keys  # 预测有、标签无 → 误标（FP）

        # 处理相同的
        for k in samekeys:
            v1 = label[k]
            v2 = res[k]
            if v1 not in dataset:
                dataset[v1] = {"FN":0,"FP":0,"TP":0} 
            if v1 == v2:
                dataset[v1]["TP"]+=1
            elif v1!=v2:
                dataset[v1]["FP"]+=1
                if v2 in dataset:
                    dataset[v2]["FN"]+=1
        for k in diffkeys1:
            v1 = label[k]
            if v1 not in dataset:
                dataset[v1] = {"FN":0,"FP":0,"TP":0} 
            dataset[v1]["FP"]+=1
        for k in diffkeys2:
            v2 = res[k]
            if isinstance(v2, str) and v2 in dataset:
                dataset[v2]["FN"]+=1
    return dataset

In [8]:
def get_f1_score(metric_dict: dict, key: str) -> float:
    """
    根据实体类型的TP/FP/FN字典，输入key计算对应F1分数
    :param metric_dict: 包含各实体类型TP/FP/FN的字典（如你提供的字典）
    :param key: 要查询的实体类型key（如'中药'/'西医诊断'）
    :return: 该类别的F1分数（保留4位小数）
    """
    # 检查key是否存在于字典中
    if key not in metric_dict:
        raise KeyError(f"Key '{key}' 不存在于字典中，可选key: {list(metric_dict.keys())}")
    
    # 提取该key对应的TP、FP、FN
    tp = metric_dict[key]['TP']
    fp = metric_dict[key]['FP']
    fn = metric_dict[key]['FN']
    
    # 计算精确率（处理分母为0的情况）
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    # 计算召回率（处理分母为0的情况）
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    # 计算F1分数（处理分母为0的情况）
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    # 保留4位小数，便于阅读
    return round(f1, 4)

In [10]:
# response_base = predict(dev_dataset, model, tokenizer)
dataset_base = f1_helper(response_base)
# 示例1：查询'中药'的F1分数
f1_herb = get_f1_score(dataset_base, '中药') # 0.2353
print(f"中药的F1分数：{f1_herb}")

# 示例2：查询'西医诊断'的F1分数
f1_west_diag = get_f1_score(dataset_base, '西医诊断') # 0.2667
print(f"西医诊断的F1分数：{f1_west_diag}")

# 示例3：查询'临床表现'的F1分数
f1_symptom = get_f1_score(dataset_base, '临床表现') # 0.0
print(f"临床表现的F1分数：{f1_symptom}")


中药的F1分数：0.3288
西医诊断的F1分数：0.0727
临床表现的F1分数：0.0645


### 2.2 加载Lora参数

In [11]:
from peft import PeftModel
# 加载LoRA权重（轻量化推理）
peft_model = PeftModel.from_pretrained(model, "./lora_weights_qwen2.5_7B", inference_mode=True)
peft_model.print_trainable_parameters()

trainable params: 0 || all params: 7,635,801,600 || trainable%: 0.0000


In [12]:
response_lora = []
x = 0
while x <len(dev_dataset):
    if x % 60 ==0:
        print("processing step:", x)
    response = predict(dev_dataset[x:x+20], peft_model, tokenizer)
    response_lora +=response
    x  = x +20
    
response_lora[:2]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


processing step: 0


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


processing step: 60


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


["�assistant\n{'活络效灵丹加味': '方剂', '当归': '中药', '丹参': '中药', '生乳香': '中药', '生没药': '中药', '柴胡': '中药', '黄芩': '中药', '大黄': '中药', '蒲公英': '中药', '甘草': '中药'}�\n�assistant\n{'活络效灵丹加味': '方剂', '当归': '中药', '丹参': '中药', '生乳香': '中药', '生没药': '中药', '柴胡': '中药', '黄芩': '中药', '大黄': '中药', '蒲公英': '中药', '甘草': '中药'}�\n�assistant\n{'活络效灵丹加味': '方剂', '当归': '中药', '丹参': '中药', '生乳香': '中药', '生没药': '中药', '柴胡': '中药', '黄芩': '中药', '大黄': '中药', '蒲公英': '中药', '甘草': '中药'}�\n�assistant\n{'活络效灵丹加味': '方剂', '当归': '中药', '丹参': '中药', '生乳香': '中药', '生没药': '中药', '柴",
 "�assistant\n{'糖尿病性功能性消化不良': '西医诊断'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}�\n�assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}�始化\n１０assistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙必利片': '西医治疗'}\tTokenNameIdentifier\n thuisontvangstassistant\n{'糖尿病性功能性消化不良': '西医诊断', '西沙

In [13]:
# response_lora = predict(dev_dataset, peft_model, tokenizer)

dataset_lora = f1_helper(response_lora)
# 示例1：查询'中药'的F1分数
f1_herb = get_f1_score(dataset_lora, '中药')
print(f"中药的F1分数：{f1_herb}")

# 示例2：查询'西医诊断'的F1分数
f1_west_diag = get_f1_score(dataset_lora, '西医诊断')
print(f"西医诊断的F1分数：{f1_west_diag}")

# 示例3：查询'临床表现'的F1分数
f1_symptom = get_f1_score(dataset_lora, '临床表现')
print(f"临床表现的F1分数：{f1_symptom}")

中药的F1分数：0.8521
西医诊断的F1分数：0.84
临床表现的F1分数：0.8148
